In [1]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from IPython.display import display, clear_output

alt.data_transformers.enable('vegafusion')

# Load the file
cat_df = pd.read_csv('../data/CAT-current-UBCBG.csv')

# Rename relevant columns
cat_df = cat_df.rename(columns={
    'TaxonName': 'Taxon',
    'ProvenanceCode': 'ProvenanceGroup',
    'ClimateRating_current': 'Current',
    'ClimateRating_emissions-limited_2050': '2050',
    'ClimateRating_business-as-usual_2090': '2090',
    'ClimateRating_bau-plus-1-degree_2090': '2090_plus_1deg'
})

# Fold Era columns into long format
long_df = cat_df.melt(
    id_vars=['ItemAccNoFull', 'ItemLocationCode', 'LocationCoordX', 'LocationCoordY', 
             'Taxon', 'LifeForm', 'ProvenanceGroup', 'LocationName'],
    value_vars=['Current', '2050', '2090', '2090_plus_1deg'],
    var_name='Era',
    value_name='ClimateRating'
)

# Filter to Main Garden only (exclude Nursery and Nitobe)
garden_df = long_df[
    (~long_df['LocationName'].str.contains('Nursery', na=False)) &
    (~long_df['LocationName'].str.contains('Nitobe', na=False))
].dropna(subset=['LocationCoordX', 'LocationCoordY'])

# Map LifeForm groups
def map_lifeform(lifeform):
    if pd.isna(lifeform):
        return 'Unknown'
    if lifeform in ['Tree', 'Shrub or Tree']:
        return 'Trees'
    if lifeform == 'Herbaceous Perennial':
        return 'Perennials'
    if lifeform == 'Annual':
        return 'Annuals'
    if lifeform == 'Bulb, Corm, or Tuber':
        return 'Bulbs'
    if lifeform in ['Shrub', 'Climber_Liana_Vine']:
        return 'Woody'
    if lifeform in ['Annual', 'Herbaceous Perennial', 'Bulb, Corm, or Tuber']:
        return 'Herbaceous'
    return 'Other'

garden_df['LifeFormGroup'] = garden_df['LifeForm'].apply(map_lifeform)


In [2]:
# Setup dropdown options

era_order = ['Current', '2050', '2090', '2090_plus_1deg']
lifeform_options = ['(All)', 'Trees', 'Perennials', 'Annuals', 'Bulbs', 'Woody', 'Herbaceous', 'Other']

era_widget = widgets.Dropdown(
    options=era_order,
    value='Current',
    description='Era:'
)

lifeform_widget = widgets.Dropdown(
    options=lifeform_options,
    value='(All)',
    description='LifeForm:'
)

provenance_widget = widgets.Dropdown(
    options=['(All)'] + sorted(garden_df['ProvenanceGroup'].dropna().unique()),
    value='(All)',
    description='Provenance:'
)

output = widgets.Output()


In [3]:
# Manually set final Garden plot boundaries
manual_long_min = -123.2525
manual_long_max = -123.2402
manual_lat_min = 49.248
manual_lat_max = 49.256

current_chart = None

def update_plot(*args):
    global current_chart
    filtered = garden_df[
        (garden_df['Era'] == era_widget.value)
    ]

    if lifeform_widget.value != '(All)':
        filtered = filtered[filtered['LifeFormGroup'] == lifeform_widget.value]

    if provenance_widget.value != '(All)':
        filtered = filtered[filtered['ProvenanceGroup'] == provenance_widget.value]

    filtered = filtered.dropna(subset=['ClimateRating'])

    chart = alt.Chart(filtered).mark_circle(size=60).encode(
        x=alt.X('LocationCoordY:Q', scale=alt.Scale(domain=[manual_long_min, manual_long_max]), axis=alt.Axis(title='Longitude')),
        y=alt.Y('LocationCoordX:Q', scale=alt.Scale(domain=[manual_lat_min, manual_lat_max]), axis=alt.Axis(title='Latitude')),
        color=alt.Color('ClimateRating:Q', scale=alt.Scale(domain=[5, 11], range=['red', 'yellow', 'green'])),
        tooltip=['Taxon:N', 'ClimateRating:Q', 'LocationCoordX:Q', 'LocationCoordY:Q']
    ).properties(
        width=600,
        height=600,
        title=f"Climate Rating Map ({era_widget.value})"
    )

    current_chart = chart

    with output:
        clear_output(wait=True)
        if not filtered.empty:
            display(chart)
        else:
            display(alt.Chart(pd.DataFrame({'x':[], 'y':[]})).mark_point())

# Hook widgets
for widget in [era_widget, lifeform_widget, provenance_widget]:
    widget.observe(update_plot, names='value')

# Layout
display(widgets.HBox([widgets.VBox([era_widget, lifeform_widget, provenance_widget]), output]))

# Initial Plot
update_plot()
